In [1]:
import os
import numpy as np
import time
import shutil
import pickle
import random
from PIL import Image, ImageOps

In [17]:
def shuffle_in_unison(a, b):
    # Gefunden bei http://stackoverflow.com/users/190280/josh-bleecher-snyder 
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype) 
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index] 
    return shuffled_a, shuffled_b

In [18]:
def move_files(input, output): 
    '''
        Input: Verzeichnis mit Bildern, wobei jede Klasse durch ein separates Verzeichnis representiert wird
        Output: Alle Bilder im Format Klasse_Nummer.jpg; Ausgabepfad ist absolut
    '''
    index = -1
    for root, dirs, files in os.walk(input):
        path = root.split('/')
        print('Gehe ins Verzeichnis ', path) 
        print('Klassen Index ', index) 
        filenum = 0

        for file in files:
            fileName, fileExtension = os.path.splitext(file) 
            if fileExtension == '.jpg':
                full_path = path[0] + '/' + path[1] + '/' + file 
                print('kopiere Quelldatei ', full_path)
                if (os.path.isfile(full_path)):
                    file = str(index) + '_ ' + path[1] + str(filenum) + fileExtension
                    print('erzeuge Zieldatei ' + output + '/' + file)
                    shutil.copy(full_path, output + '/' + file) 
                    filenum += 1
                    index += 1

In [19]:
def create_feed_files(input_path, percentage):
    '''
    Erzeugt eine train.txt und val.txt Datei fuer Caffe
    '''
    images, labels = [], [] 
    os.chdir(input_path)
    for item in os.listdir('.'):
        if not os.path.isfile(os.path.join('.', item)): continue
        try:
            label = int(item.split('_')[0]) 
            images.append(input_path+'/'+item) 
            labels.append(label)
        except:
            continue

        images = np.array(images)
    labels = np.array(labels)
    images, labels = shuffle_in_unison(images, labels) 
    X_train = images[0:int(len(images) * percentage)] 
    y_train = labels[0:int(len(labels) * percentage)] 
    X_test = images[int(len(images) * percentage):] 
    y_test = labels[int(len(labels) * percentage):] 
    trainfile = open("train.txt", "w")
    for i, l in zip(X_train, y_train): 
        trainfile.write(i + " " + str(l) + "\n")
        
    testfile = open("val.txt", "w") 
    for i, l in zip(X_test, y_test):
        testfile.write(i + " " + str(l) + "\n") 
        
    trainfile.close()    
    testfile.close()

In [20]:
def main():
    src_path = 'src_images'
    target_path = 'images'
    train_percentage = 0.85
    move_files(src_path, target_path) 
    create_feed_files(target_path, train_percentage)
    
if __name__ == "__main__": 
    main()


OSError: [Errno 2] No such file or directory: 'images'

In [21]:
%ll

total 12
drwxr-xr-x 2 root 4096 Nov 21  2016 content/
drwxr-xr-x 2 root 4096 Nov 21  2016 results/
drwxr-xr-x 2 root 4096 Nov 21  2016 style/
